In [2]:
import pandas as pd
import basedosdados as bd

In [3]:
query = '''
SELECT razao_dependencia, taxa_envelhecimento, expectativa_anos_estudo, 
    taxa_analfabetismo_18_mais, indice_gini, prop_pobreza_extrema, renda_pc,
    taxa_desocupacao_18_mais, taxa_ocupados_carteira, taxa_agua_encanada, populacao,
    populacao_urbana, populacao_rural, idhm
FROM `basedosdados.mundo_onu_adh.municipio` 
WHERE ano = 2010
LIMIT 10000
'''
df = bd.read_sql(query, billing_project_id="python-371123")


Downloading: 100%|██████████| 5565/5565 [00:01<00:00, 3085.26rows/s]


In [4]:
df.to_csv("output/psm.csv")